In [ ]:
import pandas as pd, numpy as np
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import json
import datetime

start_time = datetime.datetime.now()

# Start a new session
session = requests.Session()


# Pull the data from the Kaggle Kernels:
# iterate through each kaggle kernel in the json

# limit how many kernels pages we pull
stop_limit = 20000

# initialize the iterator to 1
i = 0

# initialize the dataframes
KernelDim = pd.DataFrame()
Bad_KernelDim = pd.DataFrame()
# Start the loop
while i < stop_limit:
    try:
        # Load the new json file
        kernel_url = "https://www.kaggle.com/kernels.json?sortBy=votes&pageSize=1&startRow=" + str(i)
        connect = session.get(kernel_url, verify=False)
        json_file = connect.json()

        # Pull all data for the kernel
        kernel_temp = pd.DataFrame.from_records(json_file)

        # if the kernel json is empty or if we hit 10000 records then stop
        if kernel_temp.empty:
            break

        # Append the new data
        KernelDim = KernelDim.append(kernel_temp)

    # if there is an exception, log it
    except Exception:
        Bad_KernelDim = Bad_KernelDim.append(pd.DataFrame(data={'url':kernel_url, 'error':'exception'}, index=[i]))
        pass
    i+=1

# Change the index to the KernelID
KernelDim = KernelDim.reset_index()
KernelDim.to_json("KernelDim.json")


## Pull the user rankings for Kernels (EDA)
#limit how many kernel rankings we pull - good idea is stop_limit_rank/20
stop_limit_ranks = max(int(stop_limit/20),1)

j = 1

# initialize the dataframes
Ranks = pd.DataFrame()
Bad_Ranks = pd.DataFrame()
while j <= stop_limit_ranks:
    try:
        # Load the new json file
        ranks_url = "https://www.kaggle.com/rankings.json?group=kernels&page=" + str(j)
        connect = session.get(ranks_url, verify=False)
        json_file = connect.json()
        
        # Pull all data for the kernel
        ranks_temp = pd.DataFrame.from_records(json_normalize(json_file).list[0])

        # If the ranks json is empty, quit
        if ranks_temp.empty:
            break

        # Append the new data
        Ranks = Ranks.append(ranks_temp)
       
    # If an exception is thrown, log it
    except Exception:
        Bad_Ranks = Bad_Ranks.append(pd.DataFrame(data={'url':ranks_url, 'error':'Exception'}, index=[j]))
        pass
    j+=1

# Change the index from joining
Ranks.set_index('userId', inplace=True)

## Pull the country for each of the ranked users
# Initialize the dataframes
UserCountry = pd.DataFrame()
Bad_UserCountry = pd.DataFrame()
# loop through each user URL
for index, row in Ranks.iterrows():
    try:
        # Load the html from the url
        country_url = "https://www.kaggle.com"+str(row.userUrl)
        country_page = session.get(country_url, verify=False)
        country_html = country_page.text
        country_soup = BeautifulSoup(country_html, "lxml")

        # Find the main content on the page
        country_json = country_soup.find('div',{'class':'site-layout__main-content'}).find('script').text
           
        # Only get the relevant html from the page
        sLoc = country_json.find("Kaggle.State.push(")+len("Kaggle.State.push(")
        eLoc = country_json.find(");performance && performance.mark")
        if sLoc == -1:
            sLoc = 0
        if eLoc == -1:
            eLoc = len(country_json)
                      
        # Trim the fat!
        country_json = country_json[sLoc:eLoc]
                     
        # Read the remaining json into a dataframe
        df = pd.read_json(country_json)

        # We only want the data from the first row. There's a lot of junk...
        d = {'userId':df.iloc[0].userId, 'country':df.iloc[0].country}
        
        # Add the data: UserID, Country
        UserCountry = UserCountry.append(pd.DataFrame(data=d, index=[0]))
            
    # Log exceptions
    except Exception:
        Bad_UserCountry = Bad_UserCountry.append(pd.DataFrame(data={'url':country_url, 'error':'Exception'}, index=[0]))
        break

# Change the index for joining
UserCountry.set_index('userId', inplace=True)
# Join
Ranks = Ranks.join(UserCountry)
Ranks.to_json("Ranks.json")

# Create a function to flatten lists
flatten = lambda l: [item for sublist in l for item in sublist]
k = 0
                           
# Initialize the packages dataframe
Packages = pd.DataFrame()
Code_Lines = pd.DataFrame()
Bad_Packages = pd.DataFrame()
# Only look at python kernels
Python_Kernels = KernelDim[KernelDim.languageName == 'Python']
# Loop through each of the kernels
for index, row in Python_Kernels.iterrows():
    try:
        # Load the HTML from the URL
        script_url = "https://www.kaggle.com"+row.scriptUrl
        script_page = session.get(script_url, verify=False)
        script_html = script_page.text
        script_soup = BeautifulSoup(script_html, "lxml")

        # Only pull html from the main content
        script_json = script_soup.find('div',{'class':'site-layout__main-content'}).find('script').text

        # Get rid of the irrelevant code
        sLoc = script_json.find("Kaggle.State.push(")+len("Kaggle.State.push(")
        eLoc = script_json.find(");performance && performance.mark")
        if sLoc == -1:
            sLoc = 0
        if eLoc == -1:
            eLoc = len(script_json)
        # Trim the fat!           
        script_json = script_json[sLoc:eLoc]

        cells = pd.DataFrame.from_records(json_normalize(json.loads(json_normalize(json.loads(script_json)).code[0])).cells[0])
        source_code = cells[cells.cell_type == 'code'].reset_index()['source']
        df = source_code.to_frame()
        df['kernel_id'] = row.id
        Code_Lines = Code_Lines.append(df)

        packages_in_snippet = list()
        for snippet in source_code:
            packages_in_line = list()
            for line in snippet:

                line = line.replace("\n","")
                # Find comments and remove everything after them
                comment_ind = line.find("#")
                if comment_ind == -1:
                    comment_ind = len(line)
                line = line[0:comment_ind]

                # if there isn't an import statement, skip it
                if line.find("import ") == -1:
                    continue
                # if the line starts with import, then split on commas and remove the import part
                elif line.find("import ") == 0:
                    line_split = line.split(",")
                    line_split[0] = line_split[0].replace("import ","").strip()
                # if the line starts with from, split it on import, remove the from, then split on commas
                elif line.find("from") == 0:
                    line_split = line.split(" import ")
                    line_split[0] = line_split[0].replace("from ","")
                    line_split[1] = line_split[1].split(", ")
                    # add in the main package (in format package.subpackage)
                    for i in range(len(line_split[1])):
                        line_split[1][i] = line_split[0]+'.'+line_split[1][i].strip()
                    line_split = line_split[1]

                # Add the packages to the list
                packages_in_line.append(line_split)
            packages_in_snippet.append(packages_in_line)

        packages_flat = flatten(flatten(packages_in_snippet))

        # separate out the aliases
        for item in packages_flat:
            item_split = item.split(" as ")
            # Pull the last item if the import uses "." as in import pandas.io.json
            last_package = item_split[0].split(".")[-1]
            # If there is no alias, reuse the package
            if len(item_split) == 1:
                d = {'package call':item_split[0], 'package':last_package, 'alias':last_package, 'kernel_id': row.id}
            else:
                d = {'package call':item_split[0], 'package':last_package, 'alias':item_split[1], 'kernel_id': row.id}

            Packages = Packages.append(pd.DataFrame(data=d, index=[k]))
            k += 1
                    
    except Exception:
        Bad_Packages = Bad_Packages.append(pd.DataFrame(data={'url':row.scriptUrl, 'error':'Exception'}, index=[0]))
        pass
    
Packages.to_json("Packages.json")



try:
    import re

    Method_Calls = pd.DataFrame()
    for index, row in Python_Kernels.iterrows():
        Kernel_Packages = Packages[Packages.kernel_id == row.id]
        kernel_source_code = Code_Lines[Code_Lines.kernel_id == row.id]['source']


        lines_with_funcs = pd.DataFrame()
        for snippet in kernel_source_code:
            for line in snippet:

                line = line.replace("\n","")
                # Find comments and remove everything after them
                comment_ind = line.find("#")
                if comment_ind == -1:
                    comment_ind = len(line)
                line = line[0:comment_ind]

                # if there IS an import statement, skip it
                if line.find("import ") >= 0:
                    continue
                else:
                    for index2, rows2 in Kernel_Packages.iterrows():
                        if line.find(str(rows2.alias)+".") >= 0:
                            d = {'call':str(rows2.alias), 'line':line, 'index_filter': rows2.kernel_id}
                            lines_with_funcs = lines_with_funcs.append(pd.DataFrame(data=d, index=[rows2.kernel_id]))
                        elif line.find(str(rows2.package)+".") >= 0:
                            d = {'call':str(rows2.package), 'line':line, 'index_filter': rows2.kernel_id}
                            lines_with_funcs = lines_with_funcs.append(pd.DataFrame(data=d, index=[rows2.kernel_id]))
                        elif line.find(str(rows2[['package call']])+".") >= 0:
                            d = {'call':str(rows2[['package call']]), 'line':line, 'index_filter': rows2.kernel_id}
                            lines_with_funcs = lines_with_funcs.append(pd.DataFrame(data=d, index=[rows2.kernel_id]))

        for index3, rows3 in lines_with_funcs.iterrows():
            line_list = re.findall(rows3.call+'\.\w+\(', rows3.line)
            for line in line_list:
                sLoc = len(str(rows3.call))+1
                eLoc = len(line)-1
                df = Kernel_Packages[Kernel_Packages.kernel_id == rows3.index_filter]
                df['method'] = line[sLoc:eLoc]
                Method_Calls = Method_Calls.append(df)
except Exception:
    pass

Method_Calls = Method_Calls.reset_index()
Method_Calls.to_json("Method_Calls.json")


end_time = datetime.datetime.now()

print("Runtime: "+str((end_time-start_time).seconds/3600)+" hours")
print(" ")
print("Kernels Parsed: "+str(len(KernelDim.index)))
print("Users Ranked: "+str(len(Ranks.index)))
print("Bad records: "+str(len(Bad_Ranks.index)+len(Bad_UserCountry.index)+len(Bad_KernelDim.index)+len(Bad_Packages.index)))